In [2]:
import sys
sys.path.append("..")
import foodcosts.core as core
import requests
import json
import calendar
import pandas as pd
from datetime import date

In [3]:
core.get_corporate_restaurants()

config.ini file available... using that


['R.E. ALLEN & OVERY',
 'R.E. LHOIST CORBAIS',
 'R.E. ZAVENTEM',
 'R.E. CEFIC',
 'R.E ALOST',
 'R.E LHOIST NIVELLES',
 'R.E LHOIST LIMELETTE',
 'BOTANIC TOWER',
 'R.E ENGIE',
 'R.E THE HIVE',
 'R.E EUROCONTROL',
 'R.E LIGHTHOUSE',
 'Immogra Park7',
 'R.E. JanDeNul',
 'ARAMIS',
 'R.E. GLAVERBEL',
 'R.E WPP']

In [4]:
# fetch hours worked for a shop
# use the API formation.fthek.be/api/hours_worked with body
# {
#	"warehouse_id": 2,
#	"yyyy_mm_dd": "2023-03-15"
# }
def get_hours_worked_for_a_shop_for_a_day(warehouse_id, yyyy_mm_dd):
#   url = "https://formation.fthek.be/api/hours_worked"
    url = "http://localhost:3000/api/hours_worked/day"
    body = {
        "warehouse_id": warehouse_id,
        "yyyy_mm_dd": yyyy_mm_dd
    }
    response = requests.get(url, json=body)
    print(response.status_code)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def get_hours_worked_for_a_shop_for_a_month_per_employee(warehouse_id, yyyy_mm):
#     url = "https://formation.fthek.be/api/hours_worked/month"
    url = "http://localhost:3000/api/hours_worked/month"
    body = {
        "warehouse_id": warehouse_id,
        "yyyy_mm": yyyy_mm
    }
    response = requests.get(url, json=body)
    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)
        print(response)
        return None

In [5]:
# call the API for a shop and a day
# return the hours worked
answer = get_hours_worked_for_a_shop_for_a_day(core.get_corporate_restaurant_id('R.E ALOST'), "2023-03-15")
print(answer)

config.ini file available... using that
200
[{'id': 2628124, 'name': 'Hans MANGELSCHOTS ', 'hours': 8.55}, {'id': 2849653, 'name': 'Bart PIRON ', 'hours': 8}, {'id': 3038011, 'name': 'Carla DE COCK', 'hours': 3.8333333333333335}]


In [6]:
# sums the hours worked for a shop
# input: [{'hours': 10.5, 'name': 'Kevin BETTE ', 'id': 2628104}, {'hours': 6.466666666666667, 'name': 'Massimo OLIVO ', 'id': 2786566}]

def sum_hours_worked_for_a_shop_and_day(warehouse_id, yyyy_mm_dd):
    hours_worked = get_hours_worked_for_a_shop_for_a_day(warehouse_id, yyyy_mm_dd)
    if hours_worked is None:
        return 0
    else:
        return sum([h['hours'] for h in hours_worked])

In [7]:
# example
sum_hours_worked_for_a_shop_and_day(core.get_corporate_restaurant_id('R.E ALOST'), "2023-03-15")

config.ini file available... using that
200


20.383333333333333

In [8]:
# returns a list of days in format yyyy-mm-dd for a given month
# input: 2023-03
def get_days_in_month(yyyy_mm):
    # Split the input string into year and month
    year, month = map(int, yyyy_mm.split('-'))

    # Get the number of days in the given month and year
    num_days = calendar.monthrange(year, month)[1]

    # Generate a list of days in the given month and year
    days = [date(year, month, day).strftime("%Y-%m-%d") for day in range(1, num_days + 1)]

    return days

# returns the list of strings (format YYYY-MM-DD) in a period (start and end dates included)
# start and end are in format YYYY-MM-DD
def get_days_in_period(start,end):
    start = date.fromisoformat(start)
    end = date.fromisoformat(end)
    delta = end - start
    dates = [start + pd.DateOffset(days=i) for i in range(delta.days + 1)]
    return [d.strftime("%Y-%m-%d") for d in dates]

#example
get_days_in_period("2023-03-01", "2023-03-04")

['2023-03-01', '2023-03-02', '2023-03-03', '2023-03-04']

In [9]:
# Example usage:
month = "2023-02"
days_in_month = get_days_in_month(month)
print(days_in_month)    



['2023-02-01', '2023-02-02', '2023-02-03', '2023-02-04', '2023-02-05', '2023-02-06', '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10', '2023-02-11', '2023-02-12', '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16', '2023-02-17', '2023-02-18', '2023-02-19', '2023-02-20', '2023-02-21', '2023-02-22', '2023-02-23', '2023-02-24', '2023-02-25', '2023-02-26', '2023-02-27', '2023-02-28']


In [10]:
# returns the total hours worked for a shop in a given month
# input: 2023-03
def get_hours_worked_for_a_shop_in_a_month(warehouse_id, yyyy_mm):
    hours_worked = get_hours_worked_for_a_shop_for_a_month_per_employee(warehouse_id, yyyy_mm)
    if hours_worked is None:
        return 0
    return sum([e['hours'] for e in hours_worked])

In [11]:
# example
hours_worked_in_alost = get_hours_worked_for_a_shop_in_a_month(core.get_corporate_restaurant_id('R.E ALOST'), "2023-02")
print(hours_worked_in_alost)

config.ini file available... using that
587.0666666666666


In [12]:
def get_hours_worked_for_a_shop_in_a_period(warehouse_id, start, end):
    days_to_consider = get_days_in_period(start, end)
    hours_worked = [get_hours_worked_for_a_shop_for_a_day(warehouse_id, day) for day in days_to_consider]
    return sum([e['hours'] for e in hours_worked])

In [13]:
# alost = core.get_sales_for_period_for_shop_per_day("2023-02-01", "2023-02-28", "R.E ALOST")
alost = core.get_sales_for_period_for_corporate_restaurant_per_day("2023-02-01", "2023-02-28", "R.E ALOST")

config.ini file available... using that


In [14]:
# sum the total sales for a shop
alost['total_sales'].sum()

57214.99726

In [15]:
core.get_corporate_restaurants()

config.ini file available... using that


['R.E. ALLEN & OVERY',
 'R.E. LHOIST CORBAIS',
 'R.E. ZAVENTEM',
 'R.E. CEFIC',
 'R.E ALOST',
 'R.E LHOIST NIVELLES',
 'R.E LHOIST LIMELETTE',
 'BOTANIC TOWER',
 'R.E ENGIE',
 'R.E THE HIVE',
 'R.E EUROCONTROL',
 'R.E LIGHTHOUSE',
 'Immogra Park7',
 'R.E. JanDeNul',
 'ARAMIS',
 'R.E. GLAVERBEL',
 'R.E WPP']

In [17]:
# foodcosts = core.get_food_costs_for_period_for_shop_per_day("2023-02-01", "2023-02-28", "La Hulpe")
foodcosts = core.get_food_costs_for_period_for_resto_per_day("2023-02-01", "2023-02-28", "R.E ALOST")

config.ini file available... using that
config.ini file available... using that


In [18]:
foodcosts.index = foodcosts['date']
foodcosts = foodcosts.drop('date', axis=1)

In [19]:
foodcosts

,atelier,supplier,total
date,,,
2023-02-01,0,181.09,181.09
2023-02-02,0,1478.15,1478.15
2023-02-03,118.42,196.51,314.93
2023-02-04,0,0,0
2023-02-05,0,0,0
2023-02-06,0,166.25,166.25
2023-02-07,25.65,1231.45,1257.10
2023-02-08,0,1727.81,1727.81
2023-02-09,0,0,0


In [20]:
foodcosts.sum()

atelier      1329.23
supplier    12060.96
total       13390.19
dtype: object

In [37]:
FULLY_LOADED_COST_HOURS = 30 

# month: 2023-02
# returns the start and end day for a given month (format YYYY-MM-DD)
def get_start_and_end_day_for_month(month):
    days_in_month = get_days_in_month(month)
    return days_in_month[0], days_in_month[-1]

def get_gross_margin_for_month(month, shop_name):
    start_date, end_date = get_start_and_end_day_for_month(month)
    foodcosts = core.get_food_costs_for_period_for_resto_per_day(start_date, end_date, shop_name)
    foodcosts.index = foodcosts['date']
    foodcosts = foodcosts.drop('date', axis=1)
    sales = core.get_sales_for_period_for_corporate_restaurant_per_day(start_date, end_date, shop_name)
    total_hours = get_hours_worked_for_a_shop_in_a_month(core.get_corporate_restaurant_id(shop_name), month)
    total_sales = sales['total_sales'].sum()
    total_foodcosts = foodcosts['total'].sum()
    gross_margin = float(total_sales) - float(total_foodcosts) - FULLY_LOADED_COST_HOURS * total_hours
    return {
        "gross_margin": gross_margin,
        "total_hours": total_hours,
        "total_sales": sales['total_sales'].sum(),
        "total_foodcosts": foodcosts['total'].sum(),
        "total_foodcosts_atelier": foodcosts['atelier'].sum(),
        "total_foodcosts_supplier": foodcosts['supplier'].sum(),
    }

In [38]:
get_gross_margin_for_month("2023-02","R.E ALOST")


config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that


{'gross_margin': 26212.807259999994,
 'total_hours': 587.0666666666666,
 'total_sales': 57214.99726,
 'total_foodcosts': Decimal('13390.19'),
 'total_foodcosts_atelier': Decimal('1329.23'),
 'total_foodcosts_supplier': Decimal('12060.96')}

In [70]:
# shop_names = core.get_corporate_restaurants()
shop_names = ['R.E. GLAVERBEL']
# shop_stats = {}
for shop_name in shop_names:
    gross_margin = get_gross_margin_for_month("2023-02", shop_name)
    print('shop ' + shop_name + ' gross margins:')
    print(gross_margin)
    shop_stats[shop_name] = gross_margin

config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
shop R.E. GLAVERBEL gross margins:
{'gross_margin': 11600.679549999993, 'total_hours': 574.8833333333334, 'total_sales': 51884.33955, 'total_foodcosts': Decimal('23037.16'), 'total_foodcosts_atelier': Decimal('12083.48'), 'total_foodcosts_supplier': Decimal('10953.68')}


In [71]:
df = pd.DataFrame.from_dict(shop_stats, orient='index').round(2)

In [72]:
display(df.sort_values('gross_margin'))

,gross_margin,total_hours,total_sales,total_foodcosts,total_foodcosts_atelier,total_foodcosts_supplier
R.E. CEFIC,-15857.62,289.80,8953.16,16116.78,12477.63,3639.15
R.E LHOIST LIMELETTE,-15176.90,202.12,3683.35,12796.75,11673.59,1123.16
R.E. LHOIST CORBAIS,-14390.05,244.17,5241.16,12306.21,10633.40,1672.81
R.E. ALLEN & OVERY,-8521.20,303.78,5894.24,5301.94,3583.22,1718.72
R.E LHOIST NIVELLES,-7921.36,133.40,1848.87,5768.23,5184.39,583.84
Immogra Park7,-6233.23,142.10,5491.49,7461.72,6824.70,637.02
ARAMIS,-2278.40,112.08,9658.66,8574.56,7591.45,983.11
R.E WPP,-813.88,143.82,7606.11,4105.49,2976.95,1128.54
R.E LIGHTHOUSE,1405.38,126.18,9685.41,4494.53,2779.05,1715.48
R.E. JanDeNul,5011.77,0.00,10261.40,5249.63,5249.63,0


In [74]:
df['gross_margin_pct'] = (df['gross_margin'] / df['total_sales'] * 100).round(0)

In [75]:
# convert decimal to float
df['total_foodcosts_atelier'] = df['total_foodcosts_atelier'].astype(float)
df['total_foodcosts_supplier'] = df['total_foodcosts_supplier'].astype(float)

In [76]:
df_pct = df.copy()
df_pct['gross_margin_pct'] = (df['gross_margin'] / df['total_sales'] * 100).round(0)
# create a dataframe with the gross margin percentage, % of foodcosts atelier and % of foodcosts supplier
df_pct['foodcosts_atelier_pct'] = (df_pct['total_foodcosts_atelier'] / df_pct['total_sales'] * 100).round(0)
df_pct['foodcosts_supplier_pct'] = (df_pct['total_foodcosts_supplier'] / df_pct['total_sales'] * 100).round(0)
df_pct['wage_cost_pct'] = (df_pct['total_hours'] * FULLY_LOADED_COST_HOURS / df_pct['total_sales'] * 100).round(0)

df_pct = df_pct[['gross_margin_pct', 'foodcosts_atelier_pct', 'foodcosts_supplier_pct', 'wage_cost_pct']]
# sort by gross margin percentage
df_pct = df_pct.sort_values(by=['gross_margin_pct'], ascending=False)
# show all columns except gross_margin_pct and wage_cost_pct

# display(df_pct[['foodcosts_atelier_pct', 'foodcosts_supplier_pct']])
display(df_pct)

,gross_margin_pct,foodcosts_atelier_pct,foodcosts_supplier_pct,wage_cost_pct
R.E. JanDeNul,49.0,51.0,0.0,0.0
R.E ALOST,46.0,2.0,21.0,31.0
R.E EUROCONTROL,38.0,26.0,15.0,22.0
R.E. ZAVENTEM,35.0,9.0,18.0,37.0
BOTANIC TOWER,24.0,11.0,31.0,33.0
R.E. GLAVERBEL,22.0,23.0,21.0,33.0
R.E THE HIVE,21.0,18.0,26.0,36.0
R.E ENGIE,19.0,27.0,23.0,31.0
R.E LIGHTHOUSE,15.0,29.0,18.0,39.0
R.E WPP,-11.0,39.0,15.0,57.0


In [77]:
!pip install tabulate

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [78]:
# create markdown table
print(df_pct.to_markdown())

|                      |   gross_margin_pct |   foodcosts_atelier_pct |   foodcosts_supplier_pct |   wage_cost_pct |
|:---------------------|-------------------:|------------------------:|-------------------------:|----------------:|
| R.E. JanDeNul        |                 49 |                      51 |                        0 |               0 |
| R.E ALOST            |                 46 |                       2 |                       21 |              31 |
| R.E EUROCONTROL      |                 38 |                      26 |                       15 |              22 |
| R.E. ZAVENTEM        |                 35 |                       9 |                       18 |              37 |
| BOTANIC TOWER        |                 24 |                      11 |                       31 |              33 |
| R.E. GLAVERBEL       |                 22 |                      23 |                       21 |              33 |
| R.E THE HIVE         |                 21 |                   

In [23]:
get_gross_margin_for_month("2023-02-01", "2023-02-28", 'PAVLOVA BAR ANVERS')

config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


{'gross_margin': -3866.1100000000006,
 'total_hours': 301.8,
 'total_sales': 8256.74,
 'total_foodcosts': 3068.85}

In [27]:
pavlova = {'gross_margin': -2357.1100000000006,
 'total_hours': 301.8,
 'total_sales': 8256.74,
 'total_foodcosts': 3068.85}